In [1]:
%reload_ext autoreload
%autoreload 2
from aldiscore.prediction.extractor import FeatureExtractor
from Bio.SeqRecord import SeqRecord
import os
from pathlib import Path
from Bio import SeqIO
from aldiscore import ROOT, get_from_config

TEST_FILE_AA = Path("data/aa.fasta")
TEST_FILE_DNA = Path("data/dna.fasta")

sequences = list(SeqIO.parse(TEST_FILE_AA, "fasta"))
print("k =", len(sequences))

k = 4


In [ ]:
import subprocess
import numpy as np
import pandas as pd

# print("Header", lines[2])
# data = np.array(lines[2:])[1:, 2:]
# occurrences = data[:, -1].astype(float)
# mask = occurrences != 0
# data = data[mask]
# occurrences = occurrences[mask]
# sort_idxs = occurrences.argsort()[::-1]
# data = data[sort_idxs]
# data
# # # print(dists.sort(axis=0))
# df = pd.DataFrame(lines[3:], columns=lines[2])
# df.Value = df.Value.map(lambda v: chr(int(v)))
# df.sort_values("Fraction", ascending=False)
# # print(chr(99))

{'FRST_File-bytes': '46', 'FRST_Entropy': '2.087233', 'FRST_Chi-square': '2859.043478', 'FRST_Mean': '102.695652', 'FRST_Monte-Carlo-Pi': '4.000000', 'FRST_Serial-Correlation': '-0.390175'}


In [58]:
import numpy as np

dummy = sequences
print(len(dummy))
extractor = FeatureExtractor(dummy)
# feat_df = extractor.compute()
extractor._init_cache()
# feats = extractor._get_ent_features()
feats = extractor.compute()
# feats.info()
feats

4


,is_dna,num_seqs,lower_bound_gap_percentage,min:js_divergence,max:js_divergence,mean:js_divergence,std:js_divergence,p5:js_divergence,p10:js_divergence,p25:js_divergence,...,max:seq_length,mean:seq_length,std:seq_length,p5:seq_length,p10:seq_length,p25:seq_length,p50:seq_length,p75:seq_length,p90:seq_length,p95:seq_length
0,False,4,0.435185,0.112407,0.26002,0.188816,0.044268,0.126132,0.139857,0.17242,...,540,305.0,195.353526,106.4,108.8,116.0,288.0,477.0,514.8,527.4


In [19]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=20)
model

ModuleNotFoundError: No module named 'sklearn'